In [1]:
!git clone https://github.com/Efromomr/mamba

Cloning into 'mamba'...
remote: Enumerating objects: 835, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 835 (delta 104), reused 58 (delta 58), pack-reused 718 (from 3)
Receiving objects: 100% (835/835), 1.58 MiB | 3.57 MiB/s, done.
Resolving deltas: 100% (474/474), done.


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [3]:
from huggingface_hub import HfApi, interpreter_login, login

In [4]:
%cd /content/mamba

/content/mamba


In [5]:
!pip install causal_conv1d --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.4 MB/s eta 0:00:00


In [6]:
!pip install . --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
!pip install datasets pytorch_lightning --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [8]:
import typing as tp

import torch
import torch.nn as nn
import transformers
from transformers import AutoTokenizer
from mamba_ssm.modules.mamba2_simple import Mamba2Simple
import matplotlib.pyplot as plt
def normalize_attn_mat(attn_mat):
    return (attn_mat - torch.min(attn_mat)) / (torch.max(attn_mat) - torch.min(attn_mat))

In [10]:
from torch.utils.data import DataLoader

from imdb_datasets import ImdbDataset
from feedforward import SwiGLU
#from layernorm import *
from model import LanguageModel
from pl_module import PLModelWrap
from mha import ModifiedMHA

In [11]:
SwiGLU_cfg = {'dim': 512, 'dim_inner': 'null', 'drop_rate': 0, 'bias': False}
Mamba_cfg = {'dim': 512, 'd_state': 16*4, 'd_conv': 4, 'expand': 2, "headdim": 4, "A_init_range": (1, 1.1)}
Attn_cfg = {'embed_dim': 512, "num_heads": 128, 'dtype': torch.float}

In [12]:
from datasets import load_dataset

In [13]:
dataset = load_dataset('stanfordnlp/imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer

tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

dataset = load_dataset('stanfordnlp/imdb')

train_dataset = ImdbDataset(notes=dataset['train']['text'],
    targets=dataset['train']['label'],
    tokenizer=tokenizer,
    max_len=512)
test_dataset = ImdbDataset(notes=dataset['test']['text'],
    targets=dataset['test']['label'],
    tokenizer=tokenizer,
    max_len=512)

train_dl = DataLoader(
        dataset=train_dataset,
        batch_size=4,
        shuffle=True,
    )

test_dl = DataLoader(
        dataset=test_dataset,
        batch_size=4,
        shuffle=False
    )

config = {'batch_size': 4,
    'epochs': 200,
    'lr': 1e-5,
    'weight_decay': 0.,
    'min_lr': 1e-4,}

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
#model.py
import typing as tp

import torch
import torch.nn as nn

from huggingface_hub import PyTorchModelHubMixin

class LanguageModel(nn.Module, PyTorchModelHubMixin):
    """
    Language model backbone.

    Args:
        dim (int): Width of the model.
        vocab_size (int): Size of the vocabulary.
        layers (list): List of layer modules.
        layer_cfgs (list): List of layer configs.
        max_length (int, optional): Maximum length of the input sequence.
        norm (nn.Module, optional): Normalization layer.
        position_embeds (tp.Callable, optional): Positional embeddings.
        embed_drop_rate (float, optional): Dropout rate for the token embeddings.
    """
    def __init__(self,
        vocab_size: int,
        layers: list,
        layer_cfgs: list,
        dim: int = 128,
        max_length: int = 512,
        norm: nn.Module = nn.RMSNorm,
        position_embeds: tp.Callable = None,
        embed_drop_rate: float = 0.0,
        *args, **kwargs
    ) -> None:
        super().__init__()

        #assert len(layer_cfgs)==len(layers), 'number of layer configs must be equal number of layers'
        #assert all(cfg['dim']==dim for cfg in layer_cfgs), 'all layer configs must specify the same dimensionality'
        #assert all(cfg['max_length']==max_length for cfg in layer_cfgs), 'all layer configs must have the same max_length'

        self.vocab_size = vocab_size
        self.token_embeds = nn.Embedding(vocab_size, dim)
        position_embeds = position_embeds(max_length, dim) if position_embeds is not None else None
        self.position_embeds = position_embeds.weight if isinstance(position_embeds, nn.Embedding) else position_embeds
        assert self.position_embeds is None or self.position_embeds.shape == (max_length, dim),\
              'position embeddings must have shape (max_length, dim)'
        self.drop_embed = nn.Dropout(embed_drop_rate)

        self.model = nn.ModuleList([])
        for layer, layer_cfg in zip(layers, layer_cfgs):
            self.model.append(nn.Sequential(norm(layer_cfg['dim'] if 'dim' in layer_cfg else layer_cfg['embed_dim']), layer(**layer_cfg)))

        self.unembed = nn.Sequential(norm(layer_cfg['dim'] if 'dim' in layer_cfg else layer_cfg['embed_dim']), nn.Linear(dim, 2))
        self.apply(self._init_weights)

    def embed(self,
        inputs_ids: torch.Tensor,
        position_ids: torch.Tensor=None
    ) -> torch.Tensor:
        batch_size, seqlen = inputs_ids.shape
        input_embeds = self.token_embeds(inputs_ids)
        if self.position_embeds is not None:
            if position_ids is None:
                position_ids = torch.arange(seqlen, dtype=torch.long)
            posembs = self.position_embeds[position_ids]
            input_embeds = input_embeds + posembs.to(input_embeds.device)
        return self.drop_embed(input_embeds)

    def forward(self, inputs_ids: torch.Tensor) -> torch.Tensor:
        x = self.embed(inputs_ids)
        for layer in self.model:
            x = x + layer(x)
        x = x.mean(dim = 1)
        return self.unembed(x)

    def _init_weights(self, m, initializer_range=0.02) -> None:
        if isinstance(m, nn.Linear):
            if m.bias is not None:
                if not getattr(m.bias, "_no_reinit", False):
                    nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, mean=0, std=initializer_range)

In [16]:
model = LanguageModel(vocab_size = tokenizer.vocab_size,
                      layers = [Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, ModifiedMHA, SwiGLU],
                      layer_cfgs = [Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Attn_cfg, SwiGLU_cfg], dim=512)


pl_model = PLModelWrap(model=model, config = config)
trainer = Trainer(max_epochs=5, accelerator = 'gpu',
                  callbacks=[pl.callbacks.EarlyStopping(monitor="test/Loss_epoch", mode="min"),
                             pl.callbacks.ModelCheckpoint(
                            save_top_k=1,
                            monitor="test/Loss_epoch",
                            mode="min",
                            dirpath=os.path.join('/content', 'checkpoints'),
                            filename="best", )])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [17]:
os.environ["TRITON_F32_DEFAULT"] = "ieee"

In [18]:
trainer.fit(pl_model, train_dl, test_dl)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/optimizer.py:378: Found unsupported keys in the optimizer configuration: {'scheduler'}
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | LanguageModel    | 26.6 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
26.6 M    Trainable params
0         Non-trainable params
26.6 M    Total params
106.309   Total estimated model params size (MB)
58        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [19]:
model_to_save = PLModelWrap.load_from_checkpoint("/content/checkpoints/best.ckpt", model=model)

In [20]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) Y


In [21]:
model_to_save.model.push_to_hub('mamba_x2_attn')

model.safetensors:   0%|          | 0.00/106M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/efromomr/mamba_x2_attn/commit/8d09125e0abb5ddf907962f9e2f3510f4b65a468', commit_message='Push model using huggingface_hub.', commit_description='', oid='8d09125e0abb5ddf907962f9e2f3510f4b65a468', pr_url=None, repo_url=RepoUrl('https://huggingface.co/efromomr/mamba_x2_attn', endpoint='https://huggingface.co', repo_type='model', repo_id='efromomr/mamba_x2_attn'), pr_revision=None, pr_num=None)

In [22]:
check_model = LanguageModel.from_pretrained('efromomr/mamba_x2_attn',
                                            layers = [Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, ModifiedMHA, SwiGLU],
                                            layer_cfgs = [Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Attn_cfg, SwiGLU_cfg])
check_model = check_model.eval()

config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/106M [00:00<?, ?B/s]

In [25]:
input_ids = torch.tensor(tokenizer.encode(dataset['test'][0]['text']))[None, :]

In [26]:
model(input_ids.to(torch.device('cuda')))

tensor([[ 3.3044, -2.1101]], device='cuda:0', grad_fn=<AddmmBackward0>)